<a href="https://colab.research.google.com/github/B-saini/AI_Project_market_trend_analysis/blob/main/Market_Trend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement




##

1.   Student Name : Bhagirath
2.   Student Code : iitrpr_ai_25010857
3.   Email : bhagirathsaini13@gmail.com
4.   Project Category : AI for Market Trend Analysis




**Overview :** I want to predict Product demand based on past sales, marketing campaign , customer behaviour and based on demand i want to update inventory so each store will have enough product in stock to meet demand but do not overload inventory as it will consume more space in store and need extra time to arrange these extra inventory

Dataset used : https://www.kaggle.com/datasets/chavindudulaj/product-demand-forecasting-dataset

# Data Cleaning and Exploration

## Data import

In [2]:
# 1. Install Kaggle Client
!pip install -q kaggle

# 2. Authenticate (Ensure kaggle.json is in /root/.kaggle/ for Colab)
# If you haven't uploaded kaggle.json yet, uncomment the lines below:
# from google.colab import files
# files.upload()

# 3. Download the specific dataset
!kaggle datasets download -d sahideseker/market-sales-forecasting-dataset

# 4. Unzip the file
!unzip -q market-sales-forecasting-dataset.zip

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
unzip:  cannot find or open market-sales-forecasting-dataset.zip, market-sales-forecasting-dataset.zip.zip or market-sales-forecasting-dataset.zip.ZIP.


# Model Training

# Model Evaluation